In [1]:
!pip install -q sqlalchemy langchain_openai langchain_community numpy


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import json
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.utilities import SQLDatabase

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

## Import data

This data set is the same data that was used by Paco Nathan in his [blog post](https://neo4j.com/developer-blog/entity-resolved-knowledge-graphs/) on entity-resolved knowledge graphs.  It is a combination of 3 files:

1. SafeGraph: a dataset of places of interest such as businesses and organizations
2. US Department of Labor Wage and Hour Compliance Action Data (DOL_WHISARD): a publicly-available dataset of labor violations of various companies
3. US Small Business Administration PPP Loans over $150K (PPP_LOANS): a publicly-available data set of Paycheck Protection Program (PPP) was a federal loans awarded to help small businesses keep their employees during the COVID-19 pandemic

These files are available for download [here](https://senzing.com/get-erkg-tutorial-evaluation-data/).

In [3]:
sg_data = []

with open('./SafeGraph_LasVegas_Partial.json') as f:
    for line in f:
        sg_data.append(json.loads(line))

sg_df = pd.DataFrame(sg_data)

dol_data = []

with open('./Dept_Labor_Whisard_LasVegas.json') as f:
    for line in f:
        dol_data.append(json.loads(line))

dol_df = pd.DataFrame(dol_data)
dol_df = dol_df.iloc[:, :13]  # Each individual infraction count is shown beyond column 13

ppp_data = []

with open('./PPP_Loans_Over_150k_LasVegas.json') as f:
    for line in f:
        ppp_data.append(json.loads(line))

ppp_df = pd.DataFrame(ppp_data)

dol_df.replace('null', np.nan, inplace=True)

sg_df.shape, dol_df.shape, ppp_df.shape

((79946, 25), (1554, 13), (3488, 19))

## Querying tabular data

I had originally tried to use the Pandas agent from LangChain but did not have much luck with it.  So I therefore elected to stand up a quickie little SQLite database.  My brief experiment here led me to conclude that the SQL agent for LangChain was a bit more reliable.

In [4]:
engine = create_engine("sqlite:///erkg_demo_sql.db")
sg_df.to_sql("sg", engine, index=False)
ppp_df.to_sql("ppp", engine, index=False)
dol_df.to_sql("dol", engine, index=False)

1554

### Verifying the SQL connection with LangChain...

In [5]:
db = SQLDatabase(engine=engine)

print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT count(*) FROM sg;"))
print(db.run("SELECT count(*) FROM ppp;"))
print(db.run("SELECT count(*) FROM dol;"))

sqlite
['dol', 'ppp', 'sg']
[(79946,)]
[(3488,)]
[(1554,)]


In [8]:
sql_agent_executor = create_sql_agent(
    ChatOpenAI(temperature=0, model="gpt-4o-mini"), 
    db=db, 
    agent_type="openai-tools", 
    verbose=True
)

## Let's get to work asking some questions!

First, let's just see that the SQL agent can handle some basic data ops questions.

In [10]:
sql_agent_executor.invoke('what tables are present?')



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


dol, ppp, sg
Invoking: `sql_db_schema` with `{'table_names': 'dol'}`



CREATE TABLE dol (
	"RECORD_TYPE" TEXT, 
	"DATA_SOURCE" TEXT, 
	"RECORD_ID" TEXT, 
	case_id TEXT, 
	"BUSINESS_NAME_ORG" TEXT, 
	"LEGAL_NAME_ORG" TEXT, 
	"BUSINESS_ADDR_LINE1" TEXT, 
	"BUSINESS_ADDR_CITY" TEXT, 
	"BUSINESS_ADDR_STATE" TEXT, 
	"BUSINESS_ADDR_POSTAL_CODE" TEXT, 
	naic_cd TEXT, 
	naics_code_description TEXT, 
	case_violtn_cnt TEXT
)

/*
3 rows from dol table:
RECORD_TYPE	DATA_SOURCE	RECORD_ID	case_id	BUSINESS_NAME_ORG	LEGAL_NAME_ORG	BUSINESS_ADDR_LINE1	BUSINESS_ADDR_CITY	BUSINESS_ADDR_STATE	BUSINESS_ADDR_POSTAL_CODE	naic_cd	naics_code_description	case_violtn_cnt
ORGANIZATION	DoL_WHISARD	53	1658108	Fabulous Freddy's (Trailwood)	Fabulous LLC	9611 Trail Wood Drive	Las Vegas	NV	89134	811192	Car Washes	0
ORGANIZATION	DoL_WHISARD	165	1419689	Boulder Station Hotel & Casino	Boulder Station, Inc.	4111 Boulder Hwy	Las Vegas	

{'input': 'what tables are present?',
 'output': 'The database contains the following tables:\n\n1. **dol**: \n   - Columns: RECORD_TYPE, DATA_SOURCE, RECORD_ID, case_id, BUSINESS_NAME_ORG, LEGAL_NAME_ORG, BUSINESS_ADDR_LINE1, BUSINESS_ADDR_CITY, BUSINESS_ADDR_STATE, BUSINESS_ADDR_POSTAL_CODE, naic_cd, naics_code_description, case_violtn_cnt.\n\n2. **ppp**: \n   - Columns: RECORD_TYPE, DATA_SOURCE, RECORD_ID, Loan_Range, BUSINESS_NAME_ORG, BUSINESS_ADDR_LINE1, BUSINESS_ADDR_CITY, BUSINESS_ADDR_STATE, BUSINESS_ADDR_POSTAL_CODE, NAICS_Code, Business_Type, OwnedByRaceEthnicity, OwnedBy, OwnedByVeteran, NonProfit, JobsReported, DateApproved, Lender, CD.\n\n3. **sg**: \n   - Columns: DATA_SOURCE, RECORD_ID, RECORD_TYPE, PLACEKEY, REL_ANCHOR_DOMAIN, REL_ANCHOR_KEY, LOCATION_NAME_ORG, BRANDS, TOP_CATEGORY, SUB_CATEGORY, NAICS_CODE, BUSINESS_GEO_LATITUDE, BUSINESS_GEO_LONGITUDE, CATEGORY_TAGS, CLOSED_ON, TRACKING_CLOSED_SINCE, PHONE_NUMBER, BUSINESS_ADDR_COUNTRY, BUSINESS_ADDR_FULL, MAILING_VE

In [11]:
sql_agent_executor.invoke('tell me about the tables in the database')



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


dol, ppp, sg
Invoking: `sql_db_schema` with `{'table_names': 'dol'}`



CREATE TABLE dol (
	"RECORD_TYPE" TEXT, 
	"DATA_SOURCE" TEXT, 
	"RECORD_ID" TEXT, 
	case_id TEXT, 
	"BUSINESS_NAME_ORG" TEXT, 
	"LEGAL_NAME_ORG" TEXT, 
	"BUSINESS_ADDR_LINE1" TEXT, 
	"BUSINESS_ADDR_CITY" TEXT, 
	"BUSINESS_ADDR_STATE" TEXT, 
	"BUSINESS_ADDR_POSTAL_CODE" TEXT, 
	naic_cd TEXT, 
	naics_code_description TEXT, 
	case_violtn_cnt TEXT
)

/*
3 rows from dol table:
RECORD_TYPE	DATA_SOURCE	RECORD_ID	case_id	BUSINESS_NAME_ORG	LEGAL_NAME_ORG	BUSINESS_ADDR_LINE1	BUSINESS_ADDR_CITY	BUSINESS_ADDR_STATE	BUSINESS_ADDR_POSTAL_CODE	naic_cd	naics_code_description	case_violtn_cnt
ORGANIZATION	DoL_WHISARD	53	1658108	Fabulous Freddy's (Trailwood)	Fabulous LLC	9611 Trail Wood Drive	Las Vegas	NV	89134	811192	Car Washes	0
ORGANIZATION	DoL_WHISARD	165	1419689	Boulder Station Hotel & Casino	Boulder Station, Inc.	4111 Boulder Hwy	Las Vegas	

{'input': 'tell me about the tables in the database',
 'output': "The database contains three tables: **dol**, **ppp**, and **sg**. Here are the details for each table:\n\n### 1. Table: dol\n- **Columns**:\n  - RECORD_TYPE\n  - DATA_SOURCE\n  - RECORD_ID\n  - case_id\n  - BUSINESS_NAME_ORG\n  - LEGAL_NAME_ORG\n  - BUSINESS_ADDR_LINE1\n  - BUSINESS_ADDR_CITY\n  - BUSINESS_ADDR_STATE\n  - BUSINESS_ADDR_POSTAL_CODE\n  - naic_cd\n  - naics_code_description\n  - case_violtn_cnt\n\n- **Sample Rows**:\n  - Fabulous Freddy's (Trailwood), Fabulous LLC, 9611 Trail Wood Drive, Las Vegas, NV, 89134, Car Washes, 0 violations.\n  - Boulder Station Hotel & Casino, Boulder Station, Inc., 4111 Boulder Hwy, Las Vegas, NV, 89121, Casino Hotels, 1 violation.\n  - MSI Landscaping, MIST Systems International, Inc., 4820 Quality Court, #B, Las Vegas, NV, 89103, Landscaping Services, 14 violations.\n\n### 2. Table: ppp\n- **Columns**:\n  - RECORD_TYPE\n  - DATA_SOURCE\n  - RECORD_ID\n  - Loan_Range\n  - BUSIN

## Now let's ask some questions that start looking at the data itself...

In [12]:
sql_agent_executor.invoke('find all references to Union Cabs in all of the tables')



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


dol, ppp, sg
Invoking: `sql_db_schema` with `{'table_names': 'dol'}`



CREATE TABLE dol (
	"RECORD_TYPE" TEXT, 
	"DATA_SOURCE" TEXT, 
	"RECORD_ID" TEXT, 
	case_id TEXT, 
	"BUSINESS_NAME_ORG" TEXT, 
	"LEGAL_NAME_ORG" TEXT, 
	"BUSINESS_ADDR_LINE1" TEXT, 
	"BUSINESS_ADDR_CITY" TEXT, 
	"BUSINESS_ADDR_STATE" TEXT, 
	"BUSINESS_ADDR_POSTAL_CODE" TEXT, 
	naic_cd TEXT, 
	naics_code_description TEXT, 
	case_violtn_cnt TEXT
)

/*
3 rows from dol table:
RECORD_TYPE	DATA_SOURCE	RECORD_ID	case_id	BUSINESS_NAME_ORG	LEGAL_NAME_ORG	BUSINESS_ADDR_LINE1	BUSINESS_ADDR_CITY	BUSINESS_ADDR_STATE	BUSINESS_ADDR_POSTAL_CODE	naic_cd	naics_code_description	case_violtn_cnt
ORGANIZATION	DoL_WHISARD	53	1658108	Fabulous Freddy's (Trailwood)	Fabulous LLC	9611 Trail Wood Drive	Las Vegas	NV	89134	811192	Car Washes	0
ORGANIZATION	DoL_WHISARD	165	1419689	Boulder Station Hotel & Casino	Boulder Station, Inc.	4111 Boulder Hwy	Las Vegas	

{'input': 'find all references to Union Cabs in all of the tables',
 'output': 'The search for references to "Union Cabs" yielded the following results:\n\n1. **From the `dol` table**: No results were found.\n2. **From the `ppp` table**: No results were found.\n3. **From the `sg` table**: One result was found:\n   - **Location Name**: Union Cabs\n   - **Brands**: []\n\nIn summary, "Union Cabs" was found only in the `sg` table.'}

In [13]:
sql_agent_executor.invoke('does Union Cabs have any violations?')



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


dol, ppp, sg
Invoking: `sql_db_schema` with `{'table_names': 'dol'}`



CREATE TABLE dol (
	"RECORD_TYPE" TEXT, 
	"DATA_SOURCE" TEXT, 
	"RECORD_ID" TEXT, 
	case_id TEXT, 
	"BUSINESS_NAME_ORG" TEXT, 
	"LEGAL_NAME_ORG" TEXT, 
	"BUSINESS_ADDR_LINE1" TEXT, 
	"BUSINESS_ADDR_CITY" TEXT, 
	"BUSINESS_ADDR_STATE" TEXT, 
	"BUSINESS_ADDR_POSTAL_CODE" TEXT, 
	naic_cd TEXT, 
	naics_code_description TEXT, 
	case_violtn_cnt TEXT
)

/*
3 rows from dol table:
RECORD_TYPE	DATA_SOURCE	RECORD_ID	case_id	BUSINESS_NAME_ORG	LEGAL_NAME_ORG	BUSINESS_ADDR_LINE1	BUSINESS_ADDR_CITY	BUSINESS_ADDR_STATE	BUSINESS_ADDR_POSTAL_CODE	naic_cd	naics_code_description	case_violtn_cnt
ORGANIZATION	DoL_WHISARD	53	1658108	Fabulous Freddy's (Trailwood)	Fabulous LLC	9611 Trail Wood Drive	Las Vegas	NV	89134	811192	Car Washes	0
ORGANIZATION	DoL_WHISARD	165	1419689	Boulder Station Hotel & Casino	Boulder Station, Inc.	4111 Boulder Hwy	Las Vegas	

{'input': 'does Union Cabs have any violations?',
 'output': 'It appears that there are no records for "Union Cabs" in the database, which means there are no violations associated with that business.'}

## Uh oh!

Houston, we have a problem!

According to these basic SQL queries, Union Cabs (an entry in the SafeGraph data) doesn't have any violations.  But if we look at the DOL data there is an entry called "Union Cab" (singular) that actually DOES have violations.   Futher, if you go into `G2Explorer.py` you will see that there are several other related cab companies with the same address like LV Cabs and Vegas Western Cabs who have additional violations.  So if you were wanting to get an accurate accounting of those violations (or many other questions you could think of to ask this data), you will miss it because you have not resolved the relationships between each of these (seemingly unrelated) cab companies!

### Now let's ask something a little more complicated...

In [14]:
sql_agent_executor.invoke('of the companies that took on the biggest loans, which had the most violations?')



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


dol, ppp, sg
Invoking: `sql_db_schema` with `{'table_names': 'dol'}`



CREATE TABLE dol (
	"RECORD_TYPE" TEXT, 
	"DATA_SOURCE" TEXT, 
	"RECORD_ID" TEXT, 
	case_id TEXT, 
	"BUSINESS_NAME_ORG" TEXT, 
	"LEGAL_NAME_ORG" TEXT, 
	"BUSINESS_ADDR_LINE1" TEXT, 
	"BUSINESS_ADDR_CITY" TEXT, 
	"BUSINESS_ADDR_STATE" TEXT, 
	"BUSINESS_ADDR_POSTAL_CODE" TEXT, 
	naic_cd TEXT, 
	naics_code_description TEXT, 
	case_violtn_cnt TEXT
)

/*
3 rows from dol table:
RECORD_TYPE	DATA_SOURCE	RECORD_ID	case_id	BUSINESS_NAME_ORG	LEGAL_NAME_ORG	BUSINESS_ADDR_LINE1	BUSINESS_ADDR_CITY	BUSINESS_ADDR_STATE	BUSINESS_ADDR_POSTAL_CODE	naic_cd	naics_code_description	case_violtn_cnt
ORGANIZATION	DoL_WHISARD	53	1658108	Fabulous Freddy's (Trailwood)	Fabulous LLC	9611 Trail Wood Drive	Las Vegas	NV	89134	811192	Car Washes	0
ORGANIZATION	DoL_WHISARD	165	1419689	Boulder Station Hotel & Casino	Boulder Station, Inc.	4111 Boulder Hwy	Las Vegas	

{'input': 'of the companies that took on the biggest loans, which had the most violations?',
 'output': 'The company that took on one of the biggest loans and had the most violations is:\n\n- **Business Name**: HSG, LLC\n- **Loan Range**: $350,000 - $1 million\n- **Number of Violations**: 36\n\nIf you need more information or additional examples, feel free to ask!'}

### Interesting.  Who are these folks?

In [15]:
sql_agent_executor.invoke('find all references to HSG, LLC')



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


dol, ppp, sg
Invoking: `sql_db_schema` with `{'table_names': 'dol'}`



CREATE TABLE dol (
	"RECORD_TYPE" TEXT, 
	"DATA_SOURCE" TEXT, 
	"RECORD_ID" TEXT, 
	case_id TEXT, 
	"BUSINESS_NAME_ORG" TEXT, 
	"LEGAL_NAME_ORG" TEXT, 
	"BUSINESS_ADDR_LINE1" TEXT, 
	"BUSINESS_ADDR_CITY" TEXT, 
	"BUSINESS_ADDR_STATE" TEXT, 
	"BUSINESS_ADDR_POSTAL_CODE" TEXT, 
	naic_cd TEXT, 
	naics_code_description TEXT, 
	case_violtn_cnt TEXT
)

/*
3 rows from dol table:
RECORD_TYPE	DATA_SOURCE	RECORD_ID	case_id	BUSINESS_NAME_ORG	LEGAL_NAME_ORG	BUSINESS_ADDR_LINE1	BUSINESS_ADDR_CITY	BUSINESS_ADDR_STATE	BUSINESS_ADDR_POSTAL_CODE	naic_cd	naics_code_description	case_violtn_cnt
ORGANIZATION	DoL_WHISARD	53	1658108	Fabulous Freddy's (Trailwood)	Fabulous LLC	9611 Trail Wood Drive	Las Vegas	NV	89134	811192	Car Washes	0
ORGANIZATION	DoL_WHISARD	165	1419689	Boulder Station Hotel & Casino	Boulder Station, Inc.	4111 Boulder Hwy	Las Vegas	

{'input': 'find all references to HSG, LLC',
 'output': 'Here are the references to "HSG, LLC" found in the database:\n\n1. From the `dol` table:\n   - **Business Name:** HSG, LLC\n   - **Legal Name:** HSG, LLC\n\n2. From the `ppp` table:\n   - **Business Name:** HSG, LLC\n   - **Address:** 4001 S. Decatur Blvd. #37-376, LAS VEGAS, NV 89103\n\nThere were no references found in the `sg` table.'}

### Some details about this

So I looked and it turns out that 4001 South Decatur Blvd is a strip mall.  And it just so happens that Suite 37 is a UPS Store, which means that this is probably a company with a PO box in that store.  In fact, if you search you will find that there are several businesses in this data set with the same address.  However, interesting things happen if you look at this entry within `G2Explorer.py`.  (See the blog post for that!)